In [1]:
import pandas as pd
import numpy as np
import copy
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
import sys
sys.path.append("/Users/cr591/OneDrive - University of Exeter/Desktop/pyCGM/pyCGM")
import metrics as cgm
import scipy.stats as stats

In [2]:
import warnings
warnings.filterwarnings('ignore')

from datetime import timedelta as time
fift_mins = time(minutes=15)
thirt_mins = time(minutes=30)

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_csv('aligned_results_rounded.csv')

In [47]:
df = df.drop_duplicates(subset=['col', 'ID', '15_min start', '15_min_end'])

In [5]:
df['lv2_false'] = df['15_min_lv2'] - df['5_min_lv2']

In [6]:
df.head()

,Unnamed: 0,col,ID,global_start,5_min start,5_min_end,5_min_diff,5_min_lv2,15_min start,15_min_end,15_min_diff,15_min_lv2,false_value,lv2_false
0,1738,akima,1001_baseline,2018-01-08 19:21:00,2018-01-08 19:21:00,2018-01-08 19:46:00,0 days 00:25:00,True,2018-01-08 19:21:00,2018-01-08 19:41:00,0 days 00:20:00,False,0,-1
1,1739,akima,1001_baseline,2018-01-10 00:41:00,2018-01-10 00:41:00,2018-01-10 01:31:00,0 days 00:50:00,True,2018-01-10 00:41:00,2018-01-10 01:31:00,0 days 00:50:00,True,0,0
2,1740,akima,1001_baseline,2018-01-10 13:01:00,2018-01-10 13:01:00,2018-01-10 13:16:00,0 days 00:15:00,False,2018-01-10 13:01:00,2018-01-10 13:21:00,0 days 00:20:00,False,0,0
3,1741,akima,1001_baseline,2018-01-11 03:06:00,2018-01-11 03:06:00,2018-01-11 03:26:00,0 days 00:20:00,False,2018-01-11 03:11:00,2018-01-11 03:26:00,0 days 00:15:00,False,0,0
4,1742,akima,1001_baseline,2018-01-11 04:16:00,2018-01-11 04:16:00,2018-01-11 04:56:00,0 days 00:40:00,True,2018-01-11 03:51:00,2018-01-11 04:56:00,0 days 01:05:00,True,0,0


In [7]:
df_min = df[['col', 'ID', '5_min_lv2', '15_min_lv2', 'false_value', 'lv2_false']]

In [8]:
df_method = df[df['col']=='cut_glc']
df_method[(df_method['5_min_lv2']==False) & (pd.isnull(df_method['15_min_lv2']))].shape[0]

28

In [21]:
def incorrect_substypes(df_method):
    lv1_lv2 = df_method[(df_method['5_min_lv2']==False) & (df_method['15_min_lv2']==True)].shape[0] # FP lv2 - was actually a lv1
    lv2_lv1 = df_method[(df_method['5_min_lv2']==True) & (df_method['15_min_lv2']==False)].shape[0] # FN lv2 - was identified as lv1
    lv1_none = df_method[(df_method['5_min_lv2']==False) & (pd.isnull(df_method['15_min_lv2']))].shape[0] # FN lv1 - was not identified as hypo
    none_lv1 = df_method[(pd.isnull(df_method['5_min_lv2'])) & (df_method['15_min_lv2']==False)].shape[0] # FP lv1 - was identified as lv1
    lv2_none = df_method[(df_method['5_min_lv2']==True) & (pd.isnull(df_method['15_min_lv2']))].shape[0] # FN lv2 - identified as none
    none_lv2 = df_method[(pd.isnull(df_method['5_min_lv2'])) & (df_method['15_min_lv2']==True)].shape[0] # FP lv2 - was none but identified as lv2
    return pd.DataFrame([[method, lv1_lv2, lv2_lv1,  lv1_none, none_lv1,lv2_none, none_lv2]])

In [29]:
results = df.groupby('col').apply(incorrect_substypes).unstack() #.reset_index().drop(columns='level_1')

In [32]:
results

,method,lv1_lv2,lv2_lv1,lv1_none,none_lv1,lv2_none,none_lv2
0,pchip,9,10,11,8,0,0
1,matern,11,6,8,14,1,0
2,polynomial_3,12,7,9,13,0,0
3,linear,3,14,17,4,0,0
4,rbf,10,6,10,15,0,0
5,akima,8,8,12,11,0,0
6,polynomial_5,12,6,8,13,0,0
7,cubicspline,12,7,9,13,0,0
8,rq,7,12,15,9,0,0
9,cut_glc,0,19,28,1,3,0


In [31]:
results = pd.DataFrame(results_list, columns=['method', 'lv1_lv2', 'lv2_lv1', 'lv1_none','none_lv1', 
                                     'lv2_none', 'none_lv2'])

In [9]:
results_list = []
for method in set(df.col.values):
    df_method = df[df['col']==method]
    lv1_lv2 = df_method[(df_method['5_min_lv2']==False) & (df_method['15_min_lv2']==True)].shape[0] # FP lv2 - was actually a lv1
    lv2_lv1 = df_method[(df_method['5_min_lv2']==True) & (df_method['15_min_lv2']==False)].shape[0] # FN lv2 - was identified as lv1
    lv1_none = df_method[(df_method['5_min_lv2']==False) & (pd.isnull(df_method['15_min_lv2']))].shape[0] # FN lv1 - was not identified as hypo
    none_lv1 = df_method[(pd.isnull(df_method['5_min_lv2'])) & (df_method['15_min_lv2']==False)].shape[0] # FP lv1 - was identified as lv1
    lv2_none = df_method[(df_method['5_min_lv2']==True) & (pd.isnull(df_method['15_min_lv2']))].shape[0] # FN lv2 - identified as none
    none_lv2 = df_method[(pd.isnull(df_method['5_min_lv2'])) & (df_method['15_min_lv2']==True)].shape[0] # FP lv2 - was none but identified as lv2
    results_list.append([method, lv1_lv2, lv2_lv1,  lv1_none, none_lv1,lv2_none, none_lv2])
results = pd.DataFrame(results_list, columns=['method', 'lv1_lv2', 'lv2_lv1', 'lv1_none','none_lv1', 
                                     'lv2_none', 'none_lv2'])

In [11]:
results['lv2_none'] + results['lv2_lv1']

0     10
1      7
2      7
3     14
4      6
5      8
6      6
7      7
8     12
9     22
10     6
dtype: int64

In [12]:
results

,method,lv1_lv2,lv2_lv1,lv1_none,none_lv1,lv2_none,none_lv2
0,pchip,9,10,11,8,0,0
1,matern,11,6,8,14,1,0
2,polynomial_3,12,7,9,13,0,0
3,linear,3,14,17,4,0,0
4,rbf,10,6,10,15,0,0
5,akima,8,8,12,11,0,0
6,polynomial_5,12,6,8,13,0,0
7,cubicspline,12,7,9,13,0,0
8,rq,7,12,15,9,0,0
9,cut_glc,0,19,28,1,3,0


52